#Introduce

This notebook is created for the case using to load the Dataset (DIV2k) with difference key, and preprocessing image:


1.   Load the dataset with key.
2.   Preprocessing image.
3.   Seperate the dataset.



In [9]:
print('import libary...')
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
  

import libary...


In [17]:
def load_dataset(key='div2k',include_validate=True):
  print('using tensorflow_datasets to load the dataset','\nloading dataset...')
  if include_validate:
    (train_ds, val_ds), metadata = tfds.load(
    key,
    split=['train','validation'],
    with_info=True,
    as_supervised=True,
   )
  else:
    (train_ds, val_ds), metadata = tfds.load(
    key,
    split=['train'],
    with_info=True,
    as_supervised=True,
    )
  print(metadata)
  return (train_ds,val_ds),metadata

In [93]:
def normalize_img(image):
  print('Normalizes images: `uint8` -> `float32`')
  return tf.cast(image, tf.float32) /255

In [14]:
def configure_performan(ds,BUFFER_SIZE=1000,BATCH_SIZE=4):
  ds=ds.cache()
  ds=ds.shuffle(buffer_size=BUFFER_SIZE)
  ds=ds.batch(BATCH_SIZE)
  ds=ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

In [ ]:
(rain_ds,val_ds),infor=load_dataset('div2k/bicubic_x4')

In [24]:

len(rain_ds)

800

In [106]:
sub_data=rain_ds.take(40)

In [88]:
for i in sub_data.take(1):
  print(i[0].shape)
  print(i[1].shape)

(306, 510, 3)
(1224, 2040, 3)


In [89]:

def show(image, label):
  plt.figure(figsize=(len(image.numpy())/80,len(image.numpy()[0])/80))
  plt.imshow(image)
  plt.title(np.array(label))
  plt.axis('off')

In [ ]:
for i in sub_data.take(2):
    # print(len(i[0].numpy()))
  show(i[0],'Low resolution')
  show(i[1],'High resolution')

In [ ]:
for i in sub_data.take(1):
  print(i[1].numpy())  

In [108]:
sub_data=sub_data.map(lambda lr,hr: (normalize_img(lr) , normalize_img(hr)),num_parallel_calls=tf.data.AUTOTUNE)

Normalizes images: `uint8` -> `float32`
Normalizes images: `uint8` -> `float32`


In [109]:
len(sub_data)

40